In [1]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Download VADER lexicon
nltk.download('vader_lexicon', quiet=True)

class SentimentAnalyzer:
    def __init__(self):
        self.sia = SentimentIntensityAnalyzer()

    def analyze_sentiment(self, text: str) -> float:
        if not text:
            return 0.0
        return self.sia.polarity_scores(text)['compound']

class TranscriptProvider:
    def __init__(self):
        self.transcripts = {
            'AAPL': "We had strong growth and positive outlook for the next quarter.",
            'GOOGL': "We faced challenges and missed our revenue targets.",
            'MSFT': "The company continues to innovate and expand market share.",
            'AMZN': "Costs have increased significantly, which will impact margins.",
            'FB': "We are optimistic about user engagement but ad revenues are down."
        }

    def get_transcript(self, ticker: str, date: pd.Timestamp) -> str:
        return self.transcripts.get(ticker, "No transcript available")

class SentimentDataProvider:
    def get_historical_sentiment(self, ticker: str, start_date: pd.Timestamp, end_date: pd.Timestamp) -> np.ndarray:
        # Placeholder: Replace with actual historical sentiment data retrieval
        return np.random.randn(100)

class EarningsCallSentimentStrategy:
    def __init__(self, sentiment_analyzer: SentimentAnalyzer, transcript_provider: TranscriptProvider, sentiment_data_provider: SentimentDataProvider):
        self.sentiment_analyzer = sentiment_analyzer
        self.transcript_provider = transcript_provider
        self.sentiment_data_provider = sentiment_data_provider

    def execute(self, date: pd.Timestamp, universe: List[str], long_threshold: float = 1.0, short_threshold: float = -1.0) -> Tuple[List[str], List[str]]:
        long_positions = []
        short_positions = []
        
        for ticker in universe:
            transcript = self.transcript_provider.get_transcript(ticker, date)
            current_sentiment = self.sentiment_analyzer.analyze_sentiment(transcript)
            
            historical_sentiment = self.sentiment_data_provider.get_historical_sentiment(ticker, date - pd.DateOffset(years=2), date)
            sentiment_zscore = (current_sentiment - np.mean(historical_sentiment)) / np.std(historical_sentiment)
            
            logging.info(f"Ticker: {ticker}, Current Sentiment: {current_sentiment:.2f}, Sentiment Z-Score: {sentiment_zscore:.2f}")
            
            if sentiment_zscore > long_threshold:
                long_positions.append(ticker)
            elif sentiment_zscore < short_threshold:
                short_positions.append(ticker)
    
        return long_positions, short_positions

def main():
    universe = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'FB']
    date = pd.Timestamp('2024-10-20')

    sentiment_analyzer = SentimentAnalyzer()
    transcript_provider = TranscriptProvider()
    sentiment_data_provider = SentimentDataProvider()

    strategy = EarningsCallSentimentStrategy(sentiment_analyzer, transcript_provider, sentiment_data_provider)
    long_positions, short_positions = strategy.execute(date, universe)

    logging.info(f"Long positions: {long_positions}")
    logging.info(f"Short positions: {short_positions}")

    # Demonstrate sentiment analysis
    for ticker, transcript in transcript_provider.transcripts.items():
        sentiment = sentiment_analyzer.analyze_sentiment(transcript)
        logging.info(f"{ticker} sentiment: {sentiment:.2f}")

if __name__ == "__main__":
    main()


2024-10-21 21:40:29,214 - INFO - Ticker: AAPL, Current Sentiment: 0.86, Sentiment Z-Score: 0.99
2024-10-21 21:40:29,215 - INFO - Ticker: GOOGL, Current Sentiment: -0.23, Sentiment Z-Score: -0.14
2024-10-21 21:40:29,216 - INFO - Ticker: MSFT, Current Sentiment: 0.77, Sentiment Z-Score: 0.72
2024-10-21 21:40:29,217 - INFO - Ticker: AMZN, Current Sentiment: 0.27, Sentiment Z-Score: 0.22
2024-10-21 21:40:29,218 - INFO - Ticker: FB, Current Sentiment: 0.39, Sentiment Z-Score: 0.34
2024-10-21 21:40:29,218 - INFO - Long positions: []
2024-10-21 21:40:29,219 - INFO - Short positions: []
2024-10-21 21:40:29,219 - INFO - AAPL sentiment: 0.86
2024-10-21 21:40:29,220 - INFO - GOOGL sentiment: -0.23
2024-10-21 21:40:29,221 - INFO - MSFT sentiment: 0.77
2024-10-21 21:40:29,222 - INFO - AMZN sentiment: 0.27
2024-10-21 21:40:29,222 - INFO - FB sentiment: 0.39
